In [2]:
import bmdrc
import pandas as pd

## Module 1: Input Data Modules 

##### LPR Class

In [3]:
from bmdrc.LPRClass import LPRClass

lpr_example = pd.read_csv("/Users/degn400/Git_Repos/bmdrc/data/LPR_Long.csv")

LPR = LPRClass(
    df = lpr_example,
    chemical = "chemical.id",
    plate = "plate.id",
    well = "well",
    concentration = "conc",
    time = "variable",
    value = "value",
    cycle_length = 20.0,
    cycle_cooldown = 10.0, 
    starting_cycle = "light"
)

...defining cycles


In [7]:
# Calculate AUC values 
aucs = LPR._df[~LPR._df["cycle"].str.contains("gap")].drop(labels = LPR._time, axis = 1)
aucs = aucs.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well, "cycle"]).sum().reset_index()



4

In [26]:
store_aucs = []

for cycle_num in range(LPR._max_cycle):

    # Pull cycle information
    cycle_num = cycle_num + 1
    light_name = "light" + str(cycle_num)
    dark_name = "dark" + str(cycle_num)

    # Merge light and dark information
    to_calc_auc = pd.merge(
        aucs[aucs["cycle"] == light_name].rename(columns = {"value":"light"}).drop("cycle", axis = 1),
        aucs[aucs["cycle"] == dark_name].rename(columns = {"value":"dark"}).drop("cycle", axis = 1),
        how = "left"
    )

    store_aucs.append([to_calc_auc["dark"][x] - to_calc_auc["light"][x] for x in range(len(to_calc_auc))])

pd.DataFrame(store_aucs).transpose()

,0,1,2,3
0,119.195001,139.074000,78.840000,108.243
1,0.000000,0.000000,0.000000,0.000
2,0.000000,0.000000,0.000000,0.000
3,44.267998,-12.235000,158.610998,141.352
4,324.989997,331.197997,228.420001,51.068
...,...,...,...,...
283,0.276000,12.953000,7.803000,-9.328
284,-4.327000,23.387000,-5.651000,12.079
285,107.622001,97.236001,32.498001,125.039
286,51.565000,67.338000,83.437000,154.584


In [10]:
aucs

,chemical.id,conc,plate.id,well,cycle,value
0,2,0.0,20544,H01,dark1,129.074001
1,2,0.0,20544,H01,dark2,140.507000
2,2,0.0,20544,H01,dark3,99.945000
3,2,0.0,20544,H01,dark4,109.586000
4,2,0.0,20544,H01,light1,9.879000
...,...,...,...,...,...,...
2299,2,100.0,20625,A12,dark4,0.000000
2300,2,100.0,20625,A12,light1,5.392000
2301,2,100.0,20625,A12,light2,34.909001
2302,2,100.0,20625,A12,light3,2.143000


In [25]:
import numpy as np

# Group by chemical, concentration, plate, well and time
LPR_grouped = LPR._df.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well, LPR._time])

# Sum times 
LPR_sum = LPR_grouped.sum().rename(columns = {LPR._time:"Sum"}).reset_index()

# Remove times with NA 
LPR_sum["Remove"] = LPR_grouped.apply(lambda df: any(np.isnan(df[LPR._value]))).reset_index().rename(columns = {0:"Remove"})["Remove"]
LPR_sum = LPR_sum[LPR_sum["Remove"] == False].drop(columns = ["Remove"])

In [34]:
# Unique and arrange times 
cycle_info = pd.DataFrame(LPR._df[LPR._time].unique()).rename({0:LPR._time}, axis = 1)
cycle_info[LPR._time] = cycle_info[LPR._time].astype(float)
cycle_info = cycle_info.sort_values(by = [LPR._time])

# Build cycle names and order. First, define all the needed variables to make this happen
cycle_order = []
first_count = 0
gap_a_count = 0
gap_b_count = 0
second_count = 0
cycle_count = 1
if LPR._starting_cycle == "light":
    other_cycle = "dark"
else:
    other_cycle = "light"

# Cycle through the light, gap, dark, and then reset
for pos in range(len(cycle_info)):
    if (first_count < LPR._cycle_length):
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count += 1
    elif (gap_a_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + LPR._starting_cycle + str(cycle_count))
        gap_a_count += 1
    elif (second_count < LPR._cycle_length):
        cycle_order.append(other_cycle + str(cycle_count))
        second_count += 1
    elif (gap_b_count < LPR._cycle_cooldown):
        cycle_order.append("gap_" + other_cycle + str(cycle_count))
        gap_b_count += 1
    else:
        cycle_count += 1
        cycle_order.append(LPR._starting_cycle + str(cycle_count))
        first_count = 1
        gap_a_count = 0
        second_count = 0
        gap_b_count = 0
    
# Add essential order information to cycle_info file
cycle_info["cycle"] = cycle_order

# Merge with data.frame
LPR._df = LPR._df.merge(cycle_info)



In [22]:
# Group by chemical, concentration, plate, and well. 
LPR_sum_grouped = LPR_sum.groupby(by = [LPR._chemical, LPR._concentration, LPR._plate, LPR._well])
